# Srock Strategy - Signal Event 0004

## 1. Import Library

In [7]:
import dai
import random
import pandas as pd

# 2. Prepare Strategy Data

In [8]:
sd = '2020-12-31'
ed = '2026-02-13'

In [9]:
ins1 = "601328.SH"
ins2 = "601998.SH"

In [10]:
sql =  f"""
WITH
data_ins1 AS (
    SELECT
        date,
        c_normalize(close/adjust_factor) AS close_ins1
    FROM cn_stock_bar1d
    QUALIFY instrument = '{ins1}'
),
data_ins2 AS (
    SELECT
        date,
        c_normalize(close/adjust_factor) AS close_ins2
    FROM cn_stock_bar1d
    QUALIFY instrument = '{ins2}'
),
data_merge1 AS (
    SELECT 
        date,
        close_ins1 - close_ins2 AS diff,
        AVG(diff)    OVER (ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS diff_mean,
        STDDEV(diff) OVER (ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS diff_sd,
        IF(diff > diff_mean + 3 * diff_sd, 1, 0) AS sign_up,
        IF(diff < diff_mean - 3 * diff_sd, 1, 0) AS sign_down,
    FROM data_ins1 JOIN data_ins2 USING (date)
),
data_merge2 AS (
    SELECT
        date,
        sign_up,
        sign_down,
        LAG(sign_up, 1)   OVER (ORDER BY date) AS sign_up_lag_1,
        LAG(sign_down, 1) OVER (ORDER BY date) AS sign_down_lag_1,
        IF(sign_up=1   AND sign_up_lag_1=0,   1, 0) AS sig1,
        IF(sign_down=1 AND sign_down_lag_1=0, 1, 0) AS sig2,
    FROM data_merge1
),
data_final AS (
    SELECT
        date,
        instrument,
        IF((instrument = '{ins1}' AND sig1 = 1) OR (instrument = '{ins2}' AND sig2 = 1), 1, 0) AS signal_buy,
        IF((instrument = '{ins1}' AND sig2 = 1) OR (instrument = '{ins2}' AND sig1 = 1), 1, 0) AS signal_sell,
    FROM cn_stock_bar1d JOIN data_merge2 USING (date)
    WHERE instrument in ('{ins1}', '{ins2}')
)
SELECT * 
FROM data_final
QUALIFY (signal_buy = 1 OR signal_sell = 1)
"""

In [11]:
df = dai.query(sql, filters={'date':[sd, ed]}).df()
df

,date,instrument,signal_buy,signal_sell
0,2021-07-13,601328.SH,0,1
1,2021-07-13,601998.SH,1,0
2,2021-09-23,601328.SH,1,0
3,2021-09-23,601998.SH,0,1
4,2022-03-30,601328.SH,0,1
5,2022-03-30,601998.SH,1,0
6,2022-07-12,601328.SH,0,1
7,2022-07-12,601998.SH,1,0
8,2022-11-23,601328.SH,0,1
9,2022-11-23,601998.SH,1,0


## 3. Backtest Strategy

In [12]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 3
    context.is_trade_by_weight = True
    context.weight = 1 / len(df['instrument'].unique())

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return

    instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
    instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])
    instruments_hold = set(context.get_account_positions().keys())

    for instrument in instruments_buy - instruments_hold:
        if context.is_trade_by_weight:
            context.order_target_percent(instrument, context.weight)
        else:
            context.order(instrument, 100)
    
    for instrument in instruments_sell:
        context.order_target_percent(instrument, 0)

    for instrument in instruments_hold:
        if (data.current_dt - context.get_position(instrument).last_sale_date).days >= context.holding_days:
            context.order_target_percent(instrument, 0)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 200000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 14:40:11] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 14:40:11] [info     ] 2021-07-13, 2025-11-24, , , instruments=2
[2026-02-18 14:40:11] [info     ] bigtrader module V2.2.0
[2026-02-18 14:40:11] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 14:40:13] [info     ] backtest done, raw_perf_ds:dai.DataSource("_fe932c52bdd8408e9e8dbe6330965402")


[2026-02-18 14:40:13] [info     ] bigtrader.v34 运行完成 [2.396s].
